# This is a python version of Michel van Biezen's tutorial on the Linear Kalman Filter
https://www.youtube.com/watch?v=CaCcOwJPytQ&t=7s


# The goal of the Kalman Filter is to estimate the state, x, of a time-variying system.

The most common example is tracking of a moving object but, the Kalman Filter has been used in a wide variety of applications including financial modeling. 

x(k) = Ax(k-1) + Bu(k-1) + w(k-1), given Z(k) = Hx(k) + v(k)  

w and k are known as process and measurment noise -- represented by Q, theprocess noise covariance matrix and R the measurement noise covariance matrix.

A is an nxn state at the previous time step (k-1) to the state of the current state
B is an nx1 matrix relateing the option control input u to the current state
z(k) - Hx(k) is the residual or measurement innnovaton; a diff or zero means that the
estimate and the measurement are equal.  

n x m matrix K is chosen to be gain or blending factor that minimizes the 
posterior error covariance.

K(k) = P(k)H.t(HPkH.t + R)-1  = Pk * H.t/(HP(k)H.t + R
                                         )
k is discrete time

K is the Kalman Grain; strikes a balance measurement and prediction

H is usually an identity matrix in the linear case; H.T is the same as H because I.T = I
R is the same form as the covariance matrix with delta(x) and delta(v) on diagonals


In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
##step 1/7;
##step 1/7
# The first step is to initialize the process covariance matrix.  The off daigonals
# can be set to zero.  The inital process covariance matrix is then updated on each
# iteration.  This step is run once, upon initialization.

def initial_process_cov_matrix():
    global P
    #initial process covariance matrix
    ##do this once
    #delta Pk = 20, 20**2 = 400
    #delta Pvk = 5; 5**2 = 25
    #20*5 cross terms
    P = np.array([[400, 100], [100, 25]])
    ##however, you can usually eliminate the off diagonals
    P[0,1] = 0
    P[1,0] = 0
    #P = np.array([[400, 0], [0, 25]])

    print("Initial Process Covariance Matrix:", P)
    
# step 2/7
# update the prediction of the state based on the previous state and elapsed time
# This step, combined with step 3, immediately below are known collectively as predict()
def update_prediction():

    global x,u,A,B
    print("dot(A,x)", np.dot(A,x))
    print("dot(B,u)", np.dot(B,u))
    
    x = np.add(np.dot(A,x), np.dot(B,u))
    ##not sure why we get x with dim 2x2 
    
    #x = x[0]
    print("x after update: ", x)


# step 3/7; update Process Covarience Matrix
# steps 3-7 are known collectively as update()

def update_predicted_process_cov_matrix():
    global A, P, Q
    #update the Process Covariance Matrix
    Q = 0.  ##reflects the error in calculating process cov matrix (not sure what)
    P = np.dot(np.dot(A, P), A.T) + Q

    #drop off diagnonals 
    P[0,1] = 0
    P[1,0] = 0
    print("Predicted Process Covariance Matrix: ", P)

# step 4/7 calculate the kalman gain, which determines how much weight is
# given to the prediction and measurement.  

def calculate_Kalman_gain():
    global R, P, H, K
    ##step 4/7 calculate the Kalman Gain
    ##use observation errors and square them
    ##delta x is 25**2=625, delta vx = 6**2=36
    S = R + np.dot(H, np.dot(P, H.T))
    K = np.dot(np.dot(P, H.T), np.linalg.inv(S))
    print("Kalman Gain Matrix: ", K)


# step 5/7 import a measurment.  The measurement can be multiplied by 
# a measurment noise array C; however, we use an identity matrix here
# Note: this step is not required in this example

def import_new_observation(m):
    global z
    #Step 5/7
    ##import the new observation
    #4260, 282
    #m = [4260, 282]
    #Observed state Yk = CYkm + Zk
    C = np.array([[1,0],[0,1]])
    measurement = np.array(m)
    z = np.dot(C, measurement)
    print("observed: ", z)

# step 6/7 update the current state
# Now that we have a prediction, a measurement and the Kalman gain,
# update the state.  This will become the current state on the next iteraton.

def update_current_state():
    global Y, H, x
    #step 6/7

    ##current state
    #Xk = Xkp + K[Yk - HXkp]
    Y = z - np.dot(H, x)
    print("Y: ", Y)
    x = x + np.dot(K, Y)
    print("adjusted x:", x)

# step 7/7 update the process covariance matrix
# Finally, we update the process covariance matrix which will become the process
# covariance matrix for the next iteration.

def update_process_cov_matrix():
    global K, H, O, R, P
    #step 7/7 update Process Covarance Matrix
    #Pk = (I-KH)*Pkp

    I = np.eye(n)
    P = np.dot(np.dot(I - np.dot(K, H), P), 
                (I - np.dot(K, H)).T) + np.dot(np.dot(K, R), K.T)   

    print("Updated Process Covariance Matrix: ", P)


In [10]:

# in this example dt = one second. 
dt=1.

# A is the state transition matrix.  This is used to update the prediction 
# based on dt
A = np.array([[1,dt], [0,1]])
n = A.shape[1]

# initialize the state matrix, which tells the position and velocity of object to track
# These values are taken from Dr. Biezen's tutorial

x = np.array([4000,280])
print(np.shape(x))

#u is the control matix--adds in the effects of acceleration
u = 2 #2 m/s**2

#B is the change in position due to acceleration
#B is a 2x1
##use [:,None]  to make b a column vector (.T also works)
B = np.array([.5*(dt**2),dt])

#R contains observation errors [delta x = 25m, delta v = 6m/s]
#these are in std; square them to get a proper R; keep fixed for this
#example
R = np.array([[625, 0], [0, 36]])
    
H = np.array([[1, 0],[0,1]])
Q = 0
#Q = np.array([[0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])

P = np.eye(n)

(2,)


In [11]:
#Vinit = 250 m/s. ; x = 400 m
#process error 20m, 5m/s error

# These values are from the tutorial
pos_measurements = [4260, 4550, 4860, 5110]
vel_measurements = [282, 285, 286, 290]

initial_process_cov_matrix()
iteration = 0
for position, velocity in zip(pos_measurements, vel_measurements):
    iteration += 1
    print("************Iteration #: {} ******************".format(iteration))
    update_prediction()
    update_predicted_process_cov_matrix()
    calculate_Kalman_gain()
    import_new_observation ([position, velocity])
    update_current_state()
    update_process_cov_matrix()
    
    
    

Initial Process Covariance Matrix: [[400   0]
 [  0  25]]
************Iteration #: 1 ******************
dot(A,x) [4280.  280.]
dot(B,u) [1. 2.]
x after update:  [4281.  282.]
Predicted Process Covariance Matrix:  [[425.   0.]
 [  0.  25.]]
Kalman Gain Matrix:  [[0.4047619  0.        ]
 [0.         0.40983607]]
observed:  [4260  282]
Y:  [-21.   0.]
adjusted x: [4272.5  282. ]
Updated Process Covariance Matrix:  [[252.97619048   0.        ]
 [  0.          14.75409836]]
************Iteration #: 2 ******************
dot(A,x) [4554.5  282. ]
dot(B,u) [1. 2.]
x after update:  [4555.5  284. ]
Predicted Process Covariance Matrix:  [[267.73028884   0.        ]
 [  0.          14.75409836]]
Kalman Gain Matrix:  [[0.29990053 0.        ]
 [0.         0.29069767]]
observed:  [4550  285]
Y:  [-5.5  1. ]
adjusted x: [4553.85054707  284.29069767]
Updated Process Covariance Matrix:  [[187.4378327    0.        ]
 [  0.          10.46511628]]
************Iteration #: 3 ******************
dot(A,x) [4838